In [1]:
!pip install --upgrade duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 7.5 MB/s eta 0:00:00:00:0100:01m

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import duckdb

In [2]:
script_ext_dir = "/opt/workspace/warc_yearly/script_extraction_out/*.parquet"
maxmind_dir = "/opt/workspace/warc_yearly/ipmaxmind_out/*.csv"

In [3]:
con = duckdb.connect("spark24.db")

In [13]:
# creating table for script extraction
con.execute(f"CREATE TABLE IF NOT EXISTS script_extraction AS SELECT * FROM read_parquet('{script_ext_dir}')")

In [61]:
script_cnt = con.execute("SELECT COUNT(*) FROM script_extraction").fetchdf()
print(script_cnt)

   count_star()
0      22127510


In [15]:
# creating table for ipmaxmind
con.execute(f"CREATE TABLE IF NOT EXISTS ipmaxmind AS SELECT * FROM read_csv_auto('{maxmind_dir}', delim='\t', ignore_errors=True)")

In [62]:
maxmind_cnt = con.execute("SELECT COUNT(*) FROM ipmaxmind").fetchdf()
print(maxmind_cnt)

   count_star()
0       9816625


In [63]:
con.execute(f"""
    CREATE TABLE IF NOT EXISTS cc24 AS
    SELECT *
    FROM script_extraction
    INNER JOIN ipmaxmind
    ON script_extraction.ip = ipmaxmind.ip
    AND script_extraction.host = ipmaxmind.host
    AND script_extraction.server = ipmaxmind.server
""")

In [64]:
result = con.execute("SELECT COUNT(*) FROM cc24").fetchdf()
print(result)

   count_star()
0       9245584


In [65]:
sample_df = con.execute("SELECT * FROM cc24 LIMIT 100000").fetchdf()
sample_df.head()

,ip,host,server,emails,script_src_attrs,ip_1,host_1,server_1,postal_code,latitude,...,accuracy_radius,continent_code,continent_name,country_iso_code,country_name,subdivision_iso_code,subdivision_name,city_name,metro_code,time_zone
0,20.25.232.11,https://www.royalcorporation.com/media/documen...,Microsoft-IIS/10.0,,,20.25.232.11,https://www.royalcorporation.com/media/documen...,Microsoft-IIS/10.0,60602,41.8874,...,20.0,NA,North America,US,United States,IL,Illinois,Chicago,602.0,America/Chicago
1,20.254.30.72,https://atletica.me/atleta/Michele-Grigoli/123889,nginx/1.18.0 (Ubuntu),,https://www.googletagmanager.com/gtag/js?id=g-...,20.254.30.72,https://atletica.me/atleta/Michele-Grigoli/123889,nginx/1.18.0 (Ubuntu),EC1N,51.5074,...,20.0,EU,Europe,GB,United Kingdom,ENG,England,London,NaN,Europe/London
2,20.26.146.238,https://cavuhb.nhs.wales/news/latest-news/mout...,S-39,,https://www.googletagmanager.com/gtag/js?id=g-...,20.26.146.238,https://cavuhb.nhs.wales/news/latest-news/mout...,S-39,EC1N,51.5074,...,200.0,EU,Europe,GB,United Kingdom,ENG,England,London,NaN,Europe/London
3,20.28.229.183,https://www.penrithcity.nsw.gov.au/services/he...,Apache/2.4.54 (Win64) OpenSSL/1.1.1s PHP/8.0.2...,mailto:council@penrith.city|mailto:council@pen...,https://www.googletagmanager.com/gtag/js?id=g-...,20.28.229.183,https://www.penrithcity.nsw.gov.au/services/he...,Apache/2.4.54 (Win64) OpenSSL/1.1.1s PHP/8.0.2...,2000,-33.8715,...,20.0,OC,Oceania,AU,Australia,NSW,New South Wales,Sydney,NaN,Australia/Sydney
4,20.29.51.63,https://www.mechanicsurplus.com/products/24812...,Apache/2.4.62 (Ubuntu),,https://images.mechanicsurplus.com/staticms/js...,20.29.51.63,https://www.mechanicsurplus.com/products/24812...,Apache/2.4.62 (Ubuntu),50307,41.6021,...,20.0,NA,North America,US,United States,IA,Iowa,Des Moines,679.0,America/Chicago


In [66]:
def contains_shopify(src_string):
    return any("shopify" in src for src in src_string.split('|'))

sample_df['is_shopify'] = sample_df['script_src_attrs'].apply(contains_shopify)
sample_df.head()

,ip,host,server,emails,script_src_attrs,ip_1,host_1,server_1,postal_code,latitude,...,continent_code,continent_name,country_iso_code,country_name,subdivision_iso_code,subdivision_name,city_name,metro_code,time_zone,is_shopify
0,20.25.232.11,https://www.royalcorporation.com/media/documen...,Microsoft-IIS/10.0,,,20.25.232.11,https://www.royalcorporation.com/media/documen...,Microsoft-IIS/10.0,60602,41.8874,...,NA,North America,US,United States,IL,Illinois,Chicago,602.0,America/Chicago,False
1,20.254.30.72,https://atletica.me/atleta/Michele-Grigoli/123889,nginx/1.18.0 (Ubuntu),,https://www.googletagmanager.com/gtag/js?id=g-...,20.254.30.72,https://atletica.me/atleta/Michele-Grigoli/123889,nginx/1.18.0 (Ubuntu),EC1N,51.5074,...,EU,Europe,GB,United Kingdom,ENG,England,London,NaN,Europe/London,False
2,20.26.146.238,https://cavuhb.nhs.wales/news/latest-news/mout...,S-39,,https://www.googletagmanager.com/gtag/js?id=g-...,20.26.146.238,https://cavuhb.nhs.wales/news/latest-news/mout...,S-39,EC1N,51.5074,...,EU,Europe,GB,United Kingdom,ENG,England,London,NaN,Europe/London,False
3,20.28.229.183,https://www.penrithcity.nsw.gov.au/services/he...,Apache/2.4.54 (Win64) OpenSSL/1.1.1s PHP/8.0.2...,mailto:council@penrith.city|mailto:council@pen...,https://www.googletagmanager.com/gtag/js?id=g-...,20.28.229.183,https://www.penrithcity.nsw.gov.au/services/he...,Apache/2.4.54 (Win64) OpenSSL/1.1.1s PHP/8.0.2...,2000,-33.8715,...,OC,Oceania,AU,Australia,NSW,New South Wales,Sydney,NaN,Australia/Sydney,False
4,20.29.51.63,https://www.mechanicsurplus.com/products/24812...,Apache/2.4.62 (Ubuntu),,https://images.mechanicsurplus.com/staticms/js...,20.29.51.63,https://www.mechanicsurplus.com/products/24812...,Apache/2.4.62 (Ubuntu),50307,41.6021,...,NA,North America,US,United States,IA,Iowa,Des Moines,679.0,America/Chicago,False


In [81]:
shopify_sites = sample_df[sample_df['is_shopify'] == True]
shopify_sites.head(15)

,ip,host,server,emails,script_src_attrs,ip_1,host_1,server_1,postal_code,latitude,...,continent_code,continent_name,country_iso_code,country_name,subdivision_iso_code,subdivision_name,city_name,metro_code,time_zone,is_shopify
1535,208.113.205.119,https://finleyhousecouture.com/products-2/loui...,Apache,,https://finleyhousecouture.com/wp-includes/js/...,208.113.205.119,https://finleyhousecouture.com/products-2/loui...,Apache,None,37.7510,...,NA,North America,US,United States,None,None,None,NaN,America/Chicago,True
4763,217.21.91.127,https://shopoutdo.com/product/iphone-series-on...,LiteSpeed,mailto:outdo@shopoutdo.com,https://shopoutdo.com/wp-includes/js/jquery/jq...,217.21.91.127,https://shopoutdo.com/product/iphone-series-on...,LiteSpeed,400017,19.0748,...,AS,Asia,IN,India,MH,Maharashtra,Mumbai,NaN,Asia/Kolkata,True
5089,23.227.38.112,https://jupitergrass.ca/products/seshgear-vers...,cloudflare,,/checkouts/internal/preloads.js?locale=en-ca|h...,23.227.38.112,https://jupitergrass.ca/products/seshgear-vers...,cloudflare,K2P,45.4130,...,NA,North America,CA,Canada,ON,Ontario,Ottawa,NaN,America/Toronto,True
5090,23.227.38.35,https://brillbirduk.co.uk/products/hypnotic-ge...,cloudflare,,//brillbirduk.co.uk/cdn/shop/t/5/assets/global...,23.227.38.35,https://brillbirduk.co.uk/products/hypnotic-ge...,cloudflare,K2P,45.4130,...,NA,North America,CA,Canada,ON,Ontario,Ottawa,NaN,America/Toronto,True
5091,23.227.38.65,https://123wow.vn/collections/thoi-trang-phu-k...,cloudflare,mailto:support@123wow.vn,https://cdnjs.cloudflare.com/ajax/libs/jquery/...,23.227.38.65,https://123wow.vn/collections/thoi-trang-phu-k...,cloudflare,K2P,45.4130,...,NA,North America,CA,Canada,ON,Ontario,Ottawa,NaN,America/Toronto,True
5092,23.227.38.66,https://2kdepartment.com/products/bape-logo-de...,cloudflare,,/checkouts/internal/preloads.js?locale=en-be|h...,23.227.38.66,https://2kdepartment.com/products/bape-logo-de...,cloudflare,K2P,45.4130,...,NA,North America,CA,Canada,ON,Ontario,Ottawa,NaN,America/Toronto,True
5093,23.227.38.70,https://2ndlifephones.com/products/iphone-x-64...,cloudflare,mailto:support@2ndlifephones.com,/checkouts/internal/preloads.js?locale=en-us|h...,23.227.38.70,https://2ndlifephones.com/products/iphone-x-64...,cloudflare,K2P,45.4130,...,NA,North America,CA,Canada,ON,Ontario,Ottawa,NaN,America/Toronto,True
5094,23.227.38.71,https://almthestudio.com/collections/necklaces...,cloudflare,,//almthestudio.com/cdn/shop/t/1/assets/lazysiz...,23.227.38.71,https://almthestudio.com/collections/necklaces...,cloudflare,K2P,45.4130,...,NA,North America,CA,Canada,ON,Ontario,Ottawa,NaN,America/Toronto,True
5095,23.227.38.72,https://bijouxkabyles.com/products/trio-kabyle...,cloudflare,mailto:contact@bijouxkabyles.com,//bijouxkabyles.com/cdn/shop/t/3/assets/theme....,23.227.38.72,https://bijouxkabyles.com/products/trio-kabyle...,cloudflare,K2P,45.4130,...,NA,North America,CA,Canada,ON,Ontario,Ottawa,NaN,America/Toronto,True
5096,23.227.38.73,https://blissed.in/products/ye-bik-gayi-hai-me...,cloudflare,,/checkouts/internal/preloads.js?locale=en-in|h...,23.227.38.73,https://blissed.in/products/ye-bik-gayi-hai-me...,cloudflare,K2P,45.4130,...,NA,North America,CA,Canada,ON,Ontario,Ottawa,NaN,America/Toronto,True


In [82]:
script_sample = shopify_sites['script_src_attrs'][8658]
script_list = script_sample.split('|')
scripts = [script for script in script_list if not script.startswith("data:")]
print(len(script_list))
print(len(scripts))
scripts

40
40


['https://buckssy.com/wp-includes/js/jquery/jquery.min.js?ver=3.7.1',
 'https://buckssy.com/wp-includes/js/jquery/jquery-migrate.min.js?ver=3.4.1',
 'https://buckssy.com/wp-content/cache/min/1/wp-content/plugins/w2s-migrate-woo-to-shopify/public/js/vi-w2s-public.js?ver=1719272908',
 'https://buckssy.com/wp-content/plugins/woocommerce/assets/js/jquery-blockui/jquery.blockui.min.js?ver=2.7.0-wc.9.1.4',
 'https://buckssy.com/wp-content/plugins/woocommerce/assets/js/frontend/add-to-cart.min.js?ver=9.1.4',
 'https://buckssy.com/wp-content/plugins/woocommerce/assets/js/zoom/jquery.zoom.min.js?ver=1.7.21-wc.9.1.4',
 'https://buckssy.com/wp-content/plugins/woocommerce/assets/js/photoswipe/photoswipe.min.js?ver=4.1.1-wc.9.1.4',
 'https://buckssy.com/wp-content/plugins/woocommerce/assets/js/photoswipe/photoswipe-ui-default.min.js?ver=4.1.1-wc.9.1.4',
 'https://buckssy.com/wp-content/plugins/woocommerce/assets/js/frontend/single-product.min.js?ver=9.1.4',
 'https://buckssy.com/wp-content/plugins/

In [11]:
def filter_and_join_scripts(script_sample):
    script_list = script_sample.split('|')
    filtered_scripts = [script for script in script_list if not script.startswith("data:")]
    return '|'.join(filtered_scripts)

In [12]:
# Step 1: Determine the chunk size
chunk_size = 100000  # You can adjust this size based on your system's memory capacity
offset = 0

# Step 2: Create a temporary table to hold the processed data
con.execute('CREATE TABLE temp_script_extraction AS SELECT * FROM script_extraction WHERE 1=0')  # Empty table with the same schema

# Step 3: Process the table in chunks
total_rows = 22127510  # Total rows in the original table

while offset < total_rows:
    # Fetch a chunk of data from the DuckDB table
    chunk_df = con.execute(f'SELECT * FROM script_extraction LIMIT {chunk_size} OFFSET {offset}').fetchdf()
    
    # Break the loop if the chunk is empty
    if chunk_df.empty:
        break

    # Step 4: Apply the function to the specific column
    chunk_df['script_src_attrs'] = chunk_df['script_src_attrs'].apply(filter_and_join_scripts)

    # Step 5: Insert the processed chunk into the temporary table
    con.register('temp_table', chunk_df)
    con.execute('INSERT INTO temp_script_extraction SELECT * FROM temp_table')

    # Increment the offset for the next chunk
    offset += chunk_size
    print(f'Processed rows: {min(offset, total_rows)} / {total_rows}')  # Optional: Print progress

# Step 6: Drop the original table and rename the temporary table
con.execute('DROP TABLE script_extraction')
con.execute('ALTER TABLE temp_script_extraction RENAME TO script_extraction')

Processed rows: 100000 / 22127510
Processed rows: 200000 / 22127510
Processed rows: 300000 / 22127510
Processed rows: 400000 / 22127510
Processed rows: 500000 / 22127510
Processed rows: 600000 / 22127510
Processed rows: 700000 / 22127510
Processed rows: 800000 / 22127510
Processed rows: 900000 / 22127510
Processed rows: 1000000 / 22127510
Processed rows: 1100000 / 22127510
Processed rows: 1200000 / 22127510
Processed rows: 1300000 / 22127510
Processed rows: 1400000 / 22127510
Processed rows: 1500000 / 22127510
Processed rows: 1600000 / 22127510
Processed rows: 1700000 / 22127510
Processed rows: 1800000 / 22127510
Processed rows: 1900000 / 22127510
Processed rows: 2000000 / 22127510
Processed rows: 2100000 / 22127510
Processed rows: 2200000 / 22127510
Processed rows: 2300000 / 22127510
Processed rows: 2400000 / 22127510
Processed rows: 2500000 / 22127510
Processed rows: 2600000 / 22127510
Processed rows: 2700000 / 22127510
Processed rows: 2800000 / 22127510
Processed rows: 2900000 / 221

In [48]:
sample_df = con.execute("SELECT * FROM script_extraction LIMIT 100000").fetchdf()
sample_df.head()

,ip,host,server,emails,script_src_attrs
0,154.197.6.73,http://020tfzc.com/index.php?type=4&i=3,Microsoft-IIS/10.0,,https://apps.bdimg.com/libs/jquery/2.1.4/jquer...
1,107.163.240.247,http://04tz.xaviergoinsphotography.com/academi...,nginx,,http://04tz.xaviergoinsphotography.com/wp-cont...
2,107.163.236.96,http://0g.ebp-online.com/product/good-as-gold-...,nginx,,
3,172.67.215.161,http://1000mm.ru/twrjmugec.html,cloudflare,,
4,160.121.229.22,http://1000piao.net/qspevdu/716-do.iunm,nginx,,//www.aliyun.com/rgn/aliyun_assets?renderer=js...


In [25]:
def contains_shopify(src_string):
    return any("shopify" in src for src in src_string.split('|'))
sample_df['contains_shopify'] = sample_df['script_src_attrs'].apply(contains_shopify)
shopify_list = sample_df[sample_df['contains_shopify'] == True]
shopify_list.head()

,ip,host,server,emails,script_src_attrs,contains_shopify
1293,35.215.179.249,http://riveraconcretecorp.com/keychron-gift-ca...,nginx,,http://www.keychron.com/cdn/shop/t/27/assets/m...,True
2935,23.227.38.65,https://1clickblinds.co.uk/collections/skyligh...,cloudflare,mailto:info@1clickblinds.co.uk,//1clickblinds.co.uk/cdn/shop/t/3/assets/lazys...,True
2987,23.227.38.32,https://3dcardmaking.com/collections/easy-3d-d...,cloudflare,,/checkouts/internal/preloads.js?locale=en-us|h...,True
2988,23.227.38.65,https://3dfilamenten.com/products/artillery-ho...,cloudflare,mailto:info@3dfilamenten.nl,/checkouts/internal/preloads.js?locale=en-gb|h...,True
3008,23.227.38.32,https://5050store.com/collections/color-navy-b...,cloudflare,,//5050store.com/cdn/shop/t/3/assets/vendor.js?...,True


In [4]:
con.execute("SHOW TABLES").fetchdf()

,name
0,cc24
1,ipmaxmind
2,script_extraction


In [6]:
con.execute("FROM duckdb_memory()").df()

,tag,memory_usage_bytes,temporary_storage_bytes
0,BASE_TABLE,1310720,0
1,HASH_TABLE,0,0
2,PARQUET_READER,0,0
3,CSV_READER,0,0
4,ORDER_BY,0,0
5,ART_INDEX,0,0
6,COLUMN_DATA,0,0
7,METADATA,0,0
8,OVERFLOW_STRINGS,0,0
9,IN_MEMORY_TABLE,0,0


In [5]:
con.execute("PRAGMA database_size").df()

,database_name,database_size,block_size,total_blocks,used_blocks,free_blocks,wal_size,memory_usage,memory_limit
0,spark24,54.7 GiB,262144,224215,224188,27,0 bytes,1.2 MiB,50.0 GiB


In [8]:
con.close()